# Microtubue Fitter

Steps:

- Discretize the initial line.
- Fit Gaussian to each points to the initial line normal vector crossing the point.
- Run linear regression on the previous fit's `mu` values.
- From those points, get the line profile with a certain thickness.
- Fit the line profile to an half Gaussian or the error function.
- Done :-)

In [1]:
%matplotlib inline

from pathlib import Path
import json
import sys
sys.path.append("../../")

import anamic

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tifffile
from scipy import ndimage

data_dir = Path('/home/hadim/Drive/Data/Fake Microtubules/anamic')

../../anamic/transformations.py:1916: UserWarning: No module named 'anamic._transformations'
  warnings.warn(str(err))


In [2]:
condition = "taper_length_nm_1500"
fnames = list((data_dir / condition).glob("*.tif"))

index = 2
fname = fnames[index]

# Open the image
im = tifffile.imread(str(fname))

# Load metadata
metadata = json.load(open(fname.with_suffix('.json')))

# Get microtubule tip coordinates
tip_start = np.array(metadata['tip_start'])
tip_end = np.array(metadata['tip_end'])